In [25]:
import pandas as pd
import censusdata
import numpy as np 
import pickle
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

## Load in census data
We'll pull some basic demographics from the American Community Survey at the block group level, so we can link it with police precinct data.

In [2]:
# pull data for just counties in NYC
nyc_counties = ['061','005','085','047','081']

results = pd.DataFrame()
for c in nyc_counties:
    county_results = censusdata.download('acs5', 2015,
                             censusdata.censusgeo([('state', '36'), ('county', c), ('block group', '*')]),
                             ['B23025_003E', 'B23025_005E', 'B15003_001E', 'B15003_002E', 'B15003_003E',
                              'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                              'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                              'B15003_014E', 'B15003_015E', 'B15003_016E']) # 'B07011_001E' - unemeployment field that's N/A
    results = results.append(county_results)

In [3]:
# clean results into final fields
results['percent_unemployed'] = results.B23025_005E / results.B23025_003E * 100
results['percent_nohs'] = (results.B15003_002E + results.B15003_003E + results.B15003_004E
                          + results.B15003_005E + results.B15003_006E + results.B15003_007E + results.B15003_008E
                          + results.B15003_009E + results.B15003_010E + results.B15003_011E + results.B15003_012E
                          + results.B15003_013E + results.B15003_014E +
                          results.B15003_015E + results.B15003_016E) / results.B15003_001E * 100
results = results[['percent_unemployed', 'percent_nohs']]
results = results.reset_index()
results.head()

,index,percent_unemployed,percent_nohs
0,"Block Group 1, Census Tract 13, New York Count...",2.27,0.89
1,"Block Group 2, Census Tract 13, New York Count...",4.10,0.00
2,"Block Group 1, Census Tract 14.01, New York Co...",1.76,6.65
3,"Block Group 2, Census Tract 14.01, New York Co...",1.86,7.27
4,"Block Group 1, Census Tract 14.02, New York Co...",4.10,20.16


In [4]:
# create blockgroup IDs (for merging) and add to the results dataframe
blockgroups = []
for i in range(0,len(results)):
    geoid = results['index'][i].geo[0][1] + results['index'][i].geo[1][1] + results['index'][i].geo[2][1] + results['index'][i].geo[3][1]
    blockgroups.append(geoid)

results['blockgroup'] = blockgroups
results['blockgroup'] = results['blockgroup'].astype('int')
results.head()
    

,index,percent_unemployed,percent_nohs,blockgroup
0,"Block Group 1, Census Tract 13, New York Count...",2.27,0.89,360610013001
1,"Block Group 2, Census Tract 13, New York Count...",4.10,0.00,360610013002
2,"Block Group 1, Census Tract 14.01, New York Co...",1.76,6.65,360610014011
3,"Block Group 2, Census Tract 14.01, New York Co...",1.86,7.27,360610014012
4,"Block Group 1, Census Tract 14.02, New York Co...",4.10,20.16,360610014021


## Load precinct data and merge to create final dataset
I'll use [this key](https://johnkeefe.net/nyc-police-precinct-and-census-data) from John Keefe that links census blocks to police precincts

In [6]:
precincts = pd.read_csv('precinct_block_key_2020.csv')
precincts.head()

,geoid10,precinct_2020
0,360050001000001,NaN
1,360050001000002,NaN
2,360050001001000,114.0
3,360050001001001,114.0
4,360050001001002,114.0


In [7]:
# load in block <--> precinct crosswalk
precincts = pd.read_csv('precinct_block_key_2020.csv')

# the file gives us blocks, we'll turn these into block groups to match census data
precincts['blockgroup'] = precincts['geoid10'].astype(str).str[:-3].astype(np.int64)
precincts.head()

,geoid10,precinct_2020,blockgroup
0,360050001000001,NaN,360050001000
1,360050001000002,NaN,360050001000
2,360050001001000,114.0,360050001001
3,360050001001001,114.0,360050001001
4,360050001001002,114.0,360050001001


In [8]:
# add precinct to the results dataframe
results = pd.merge(left=results,right=precincts,how='left',on='blockgroup')

,index,percent_unemployed,percent_nohs,blockgroup,geoid10,precinct_2020
39144,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001001,104.0
39145,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001002,104.0
39146,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001003,104.0


In [10]:
results.tail()

,index,percent_unemployed,percent_nohs,blockgroup,geoid10,precinct_2020
39142,"Block Group 2, Census Tract 598, Queens County...",4.10,14.13,360810598002,360810598002006,105.0
39143,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001000,104.0
39144,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001001,104.0
39145,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001002,104.0
39146,"Block Group 1, Census Tract 599, Queens County...",12.28,13.74,360810599001,360810599001003,104.0


In [14]:
# groupby precinct, taking average and median of our 
results.groupby('precinct_2020').agg({'percent_unemployed': ['mean','median'],'percent_nohs': ['mean','median']}).head()

percent_unemployed        percent_nohs       
                            mean median         mean median
precinct_2020                                              
1.0                         3.49   2.80         2.82   1.45
5.0                         6.85   6.49        31.23  28.88
6.0                         4.65   4.06         1.01   0.06
7.0                         8.51   7.38        27.21  26.00
9.0                         7.00   5.26        13.85   7.55

In [23]:
# create dataframe from groupby
census_df = results.groupby('precinct_2020').agg({'percent_unemployed': ['mean','median'],'percent_nohs': ['mean','median']})

# rename columns and reset index
census_df.columns = ['percent_unemployed_mean','percent_unemployed_median',
                    'percent_nohs_mean', 'percent_nohs_median']
census_df = census_df.reset_index()
census_df.head()

,precinct_2020,percent_unemployed_mean,percent_unemployed_median,percent_nohs_mean,percent_nohs_median
0,1.0,3.49,2.80,2.82,1.45
1,5.0,6.85,6.49,31.23,28.88
2,6.0,4.65,4.06,1.01,0.06
3,7.0,8.51,7.38,27.21,26.00
4,9.0,7.00,5.26,13.85,7.55


## Export final dataset as pickle

In [26]:
with open('final_census_data.pickle', 'wb') as to_write:
    pickle.dump(census_df, to_write)